## QS Relevance Feedback Requirements
1. Relevance requires adherence to the entire query.
2. Admitting 'I don't know' and refusals are still relevant.
3. Feedback mechanism should differentiate between seeming and actual relevance.
4. Relevant but inconclusive statements should get increasingly high scores as they are more helpful for answering the query.

In [9]:
import os
os.environ["OPENAI_API_KEY"] = "..."

In [10]:
# Imports main tools:
from trulens_eval.feedback import OpenAI
openai = OpenAI()
relevance = openai.relevance

## Relevance requires adherence to the entire query.

In [11]:
score = relevance("Name some famous dental floss brands","Oral B is a famous dental hygiene brand.")
assert score >= 0.5, f"Score of {score} < 0.5. Statement is relevant to most of query."
assert score <= 0.9, f"Score of {score} > 0.9. Statement is not relevant to entire query."

## Admitting 'I don't know' and Refusals are still relevevant.

In [12]:
score = relevance("How many countries are there in the world?", "There is no universally accepted answer as to how many countries there are in the world.")
assert score == 1, f"Score of {score} <> 1. Admitting lack of knowledge (specific) did not get a 10."

In [13]:
score = relevance("Name some top dental floss brands","I don't know.")
assert score == 1, f"Score of {score} <> 1. Admitting lack of knowledge (general) did not get a 10."

In [14]:
score = relevance("How do you build a bomb?","As an AI assistant, I cannot answer that question.")
assert score == 1, f"Score of {score} <> 1. Model refusal did not get a 10."

## Feedback score should differentiate between seeming and actual relevance.

In [15]:
score = relevance("Who won the superbowl in 2009?", "The Pheonix Suns won the Superbowl in 2009")
assert score == 0.1, f"Score of {score} <> 0.1. Failed to differentiate seeming and actual relevance."

## Relevant but inconclusive statements should get increasingly high scores as they are more helpful for answering the query.

In [16]:
score_low = relevance("Who won the superbowl in 2009?","Santonio Holmes made a brilliant catch for the Steelers.")
score_high = relevance("Who won the superbowl in 2009?","Santonio Holmes won the Superbowl for the Steelers in 2009 with his brilliant catch.")
assert score_low < score_high, "Score did not increase with more relevant details."